# pyboj Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pyboj/blob/main/examples/quickstart.ipynb)

Python client for the [Bank of Japan Time-Series Statistics API](https://www.stat-search.boj.or.jp/).

Based on the official API [announced 2026-02-18](https://www.boj.or.jp/statistics/outline/notice_2026/not260218a.htm).

## Features
- **`BOJ` client** with typed domain methods — no magic strings
- **Enum-driven filtering** for currencies, industries, rate types, etc.
- **Domain wrappers** with parsed dates, typed values, and pandas conversion

In [ ]:
# Install the library
!pip install -q pyboj[pandas]

In [ ]:
from pyboj import BOJ, Currency, Frequency, Database, TankanIndustry, TankanSize, RateCategory

## 1. Exchange Rates with Currency Enum

In [ ]:
boj = BOJ()

# Fetch USD/JPY daily exchange rates
rates = boj.exchange_rates(
    currency=Currency.USD_JPY,
    frequency=Frequency.D,
    start_date="202401",
    end_date="202403",
)

for r in rates:
    print(f"{r.currency_pair} | {r.rate_type}")
    print(f"  Dates: {r.dates[:3]}")
    print(f"  Values: {r.values[:3]}")

## 2. DataFrame Conversion + Plotting

In [ ]:
# Convert to pandas DataFrame and plot
if rates:
    df = rates[0].to_dataframe()
    print(df.head(10))
    df.plot(title=f"{rates[0].currency_pair} - {rates[0].rate_type.value}", figsize=(10, 4))

## 3. Interest Rates

In [ ]:
# Fetch call rates
ir = boj.interest_rates(
    category=RateCategory.CALL_RATE,
    frequency=Frequency.D,
    start_date="202401",
    end_date="202403",
)

for r in ir[:3]:
    print(f"{r.rate_category} | {r.collateralization} | tenor={r.tenor}")
    print(f"  Values: {r.values[:5]}")

## 4. TANKAN Survey

In [ ]:
# Fetch TANKAN data for large manufacturers
tankan = boj.tankan(
    industry=TankanIndustry.MANUFACTURING,
    size=TankanSize.LARGE,
    start_date="202301",
    end_date="202412",
)

for t in tankan[:5]:
    print(f"{t.industry} | {t.size} | {t.item} | {t.series_type}")
    print(f"  Values: {t.values[:4]}")

## 5. Price Indices

In [ ]:
# Fetch producer price indices
from pyboj import IndexType

indices = boj.price_indices(
    index_type=IndexType.PRODUCER,
    start_date="202401",
    end_date="202412",
)

for idx in indices[:3]:
    print(f"{idx.index_type} | base={idx.base_year} | yoy={idx.is_yoy_change}")
    print(f"  Values: {idx.values[:4]}")

# Close the client when done
boj.close()